In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
import pandas as pd
import MetaTrader5 as mt5
from datetime import datetime

In [2]:
def mt5connect():
    account = int(25017742)

    mt5.initialize()
    authorized=mt5.login(account, password="O6!qSze6yBX&", server="TickmillUK-Demo")
    #authorized=mt5.login(account)

    if authorized:
        print("Connected: Connecting to MT5 Client")
    else:
        print("Failed to connect at account #{}, error code: {}"
              .format(account, mt5.last_error()))
mt5connect()

Connected: Connecting to MT5 Client


In [3]:
driver = 0
def start_chrome():
    global driver
    options = webdriver.ChromeOptions()
    options.add_argument('user-data-dir=C:\\Users\\Root\\AppData\\Local\\Google\\Chrome\\User Data')
    driver = webdriver.Chrome(executable_path=r'chromedriver.exe', options=options)
    driver.get("https://www.fxautotrade.com/login")
    driver.find_elements(By.CLASS_NAME, 'btn-google')[0].click()
    time.sleep(5)
    driver.get("https://www.fxautotrade.com/manage/4170848")
    time.sleep(5)
    
start_chrome()

In [4]:
def now():
    now = datetime.now()
    dt_string = now.strftime("%d/%m/%Y %H:%M:%S")
    return dt_string

def positions_get(symbol=None):
    if(symbol is None):
	    res = mt5.positions_get()
    else:
        res = mt5.positions_get(symbol=symbol)

    if(res is not None and res != ()):
        df = pd.DataFrame(list(res),columns=res[0]._asdict().keys())
        df['time'] = pd.to_datetime(df['time'], unit='s')
        return df
    
    return pd.DataFrame()

def close_position(deal_id):
    open_positions = positions_get()
    
    open_positions = open_positions[open_positions['ticket'] == deal_id]
    order_type  = list(open_positions["type"])[0]
    symbol = list(open_positions['symbol'])[0]
    volume = list(open_positions['volume'])[0]

    if(order_type == mt5.ORDER_TYPE_BUY):
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    else:
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
	
    close_request={
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": float(volume),
        "type": order_type,
        "position": deal_id,
        "price": price,
        "magic": 234000,
        "comment": "Close trade",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }

    result = mt5.order_send(close_request)
    
    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(now(), "Failed to close order :(")
    else:
        print (now(), "Order successfully closed!")


def open_position(pair, order_type, size, tp=None, stop=None):
    symbol_info = mt5.symbol_info(pair)
    if symbol_info is None:
        print(now(), pair, "not found")
        return

    if not symbol_info.visible:
        print(now(), pair, "is not visible, trying to switch on")
        if not mt5.symbol_select(pair, True):
            print(now(), "symbol_select({}}) failed, exit",pair)
            return
    print(now(), pair, "found!")

    point = symbol_info.point
    
    sl = stop
    tp = tp
        
    if(order_type == "BUY"):
        order = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(pair).ask
        
            
    if(order_type == "SELL"):
        order = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(pair).bid
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": pair,
        "volume": float(size),
        "type": order,
        "price": price,
        "magic": 234000,
        "comment": "",
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    
    if stop:
        request["sl"] = sl
    if tp:
        request["tp"] = tp

    result = mt5.order_send(request)
    #print(result)

    if result.retcode != mt5.TRADE_RETCODE_DONE:
        print(now(), "Failed to send order :(")
        print(now(), mt5.last_error())
        print(now(), result)
        return -1
    else:
        print (now(), "Order successfully placed!")
        
        
    
    return result

In [5]:
trades_string = []
class trade:
    def __init__(self, t_id, date, time, side, lot, symbol, o, sl, tp):
        self.t_id = t_id
        self.date = date
        self.time = time
        self.side = side
        self.lot = lot
        self.symbol = symbol
        self.executed = False
        self.closed = False
        
        self.o = None
        self.sl = None
        self.tp = None
        
        if o != "-":
            self.o = float(o)
        if sl != "-":
            self.sl = float(sl)
        if tp != "-":
            self.tp = float(tp)
        

    def enter(self):
        if self.executed:
            print(now(), "already executed!")
            return
        
        self.res = open_position(self.symbol,self.side.upper(), max(round(float(self.lot)*2*100)/100, 0.01))
        self.executed = True
        #print(self.res.order)
        
    def close(self):
        if self.closed:
            print(now(), "already closed")
                                 
        print(now(), "close", self.t_id)
        close_position(self.res.order)
        self.closed = True
        
    closed = False
    executed = False
        
    t_id = 0
    date = ""
    time = ""
    side = ""
    lot = ""
    symbol = ""
    o = 0
    sl = 0
    tp = 0

    
trades = []

In [6]:
def update_trades_string():
    global trades_string
    table = driver.find_elements(By.ID, "openTradesCont")[0]
    tr_elements = table.find_elements_by_tag_name("tr")
    trades_string = []
    for i in tr_elements:
        t = i.text
        if "Buy" in t or "Sell" in t:
            trades_string.append(t)

In [7]:
def parse_strings():
    global trades
    new_trades = []
    for i in trades_string:
        t_id, date, time, side, lot, symbol, o, sl, tp, current, _, _, _ = i.split()
        new_trades.append(trade(t_id, date, time, side, lot, symbol, o, sl, tp))
        
    for new_trade in new_trades:
        trade_there = False
        for old_trade in trades:
            if old_trade.t_id == new_trade.t_id:
                trade_there = True
        
        if not trade_there:
            new_trade.enter()
            trades.append(new_trade)
            
    
    while True:
        sth = False
        for index, old_trade in enumerate(trades):
            trade_still_there = False
            for new_trade in new_trades:
                if old_trade.t_id == new_trade.t_id:
                    trade_still_there = True
                
            if not trade_still_there:
                old_trade.close()
                del trades[index]
                sth = True
                break
        
        if not sth:
            break

In [8]:

last_trades_open = 0
last_positions_in_mt5 = 0
def validate_positions():
    global last_trades_open
    global last_positions_in_mt5
    
    if last_trades_open != len(trades):
        last_trades_open = len(trades)
        print(now(), last_trades_open, "trades registered")
    
    positions = mt5.positions_get()
    
    if last_positions_in_mt5 != len(positions):
        last_positions_in_mt5 = len(positions)
        print(now(), last_positions_in_mt5, "positions in mt5 found")
        
        
    for index, i in enumerate(trades):
        is_there = False
        for o in positions:
            if i.res.order == o.ticket:
                is_there = True
        if not is_there:
            del trades[index]
            #print("missing position", i.res.order, "- deleted from list")
            validate_positions()
            break
            
last_pl = 0
last_change_seen = 0
def check_still_alive():
    global last_pl
    global last_change_seen
    text = driver.find_element(By.ID, "floatingPl").text
    if text != last_pl:
        last_pl = text
        last_change_seen = time.time()
        
    if text == last_pl:
        if time.time() - last_change_seen >= 60*3:
            last_change_seen = time.time()
            print(now(), "no change detected for 3 minutes - reloading page")
            driver.get("https://www.fxautotrade.com/manage/4170848")
            time.sleep(5)
    
            

            
    
def update():
    check_still_alive()
    validate_positions()
    update_trades_string()
    parse_strings()
    

In [9]:
print("starting update loop...")
while True:
    try:
        time.sleep(0.1)
        update()
    except KeyboardInterrupt:
        print("break")
        break
    except Exception as e:
        print(repr(e))
        print("restarting in 2 seconds...")
        time.sleep(2)
        try:
            driver.quit()
            start_chrome()
            mt5connect()
        except Exception as e:
            print(repr(e))
        

starting update loop...
07/06/2022 07:44:04 EURUSD found!
07/06/2022 07:44:04 Order successfully placed!
07/06/2022 07:44:04 EURUSD found!
07/06/2022 07:44:04 Order successfully placed!
07/06/2022 07:44:04 EURCAD found!
07/06/2022 07:44:04 Order successfully placed!
07/06/2022 07:44:04 EURUSD found!
07/06/2022 07:44:04 Order successfully placed!
07/06/2022 07:44:04 EURUSD found!
07/06/2022 07:44:05 Order successfully placed!
07/06/2022 07:44:05 GBPUSD found!
07/06/2022 07:44:05 Order successfully placed!
07/06/2022 07:44:05 USDCAD found!
07/06/2022 07:44:05 Order successfully placed!
07/06/2022 07:44:05 USDCAD found!
07/06/2022 07:44:05 Order successfully placed!
07/06/2022 07:44:05 GBPUSD found!
07/06/2022 07:44:05 Order successfully placed!
07/06/2022 07:44:05 9 trades registered
07/06/2022 07:44:05 9 positions in mt5 found
break


In [10]:
def close_all_trades():
    while len(trades) > 0:
        trades[0].close()
        del trades[0]
        
close_all_trades()

07/06/2022 07:56:12 close 1000029
07/06/2022 07:56:12 Order successfully closed!
07/06/2022 07:56:12 close 1000030
07/06/2022 07:56:12 Order successfully closed!
07/06/2022 07:56:12 close 1000031
07/06/2022 07:56:12 Order successfully closed!
07/06/2022 07:56:12 close 1000032
07/06/2022 07:56:12 Order successfully closed!
07/06/2022 07:56:12 close 1000033
07/06/2022 07:56:12 Order successfully closed!
07/06/2022 07:56:12 close 1000038
07/06/2022 07:56:12 Order successfully closed!
07/06/2022 07:56:12 close 1000035
07/06/2022 07:56:12 Order successfully closed!
07/06/2022 07:56:12 close 1000036
07/06/2022 07:56:12 Order successfully closed!
07/06/2022 07:56:12 close 1000037
07/06/2022 07:56:12 Order successfully closed!


In [11]:
driver.quit()